In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
from os.path import expanduser
## actions required!!!!!!!!!!!!!!!!!!!! change your folder path 
path_repo = expanduser("~/Documents/G3_2/regime-identification"); sys.path.append(path_repo)
path_file = expanduser("~/data/G3_2/regime-identification/simulation")
path = {}
for folder in ["data", "estimation", "score", "figure", "latex"]:
    path[folder] = f"{path_file}/{folder}"

job_id = 0 #int(os.environ["SLURM_ARRAY_TASK_ID"])

In [2]:
from numpy.random import RandomState

In [3]:
from regime.jump import *
from regime.simulation_helper import *

# 3-model-fitting

In this notebook we fit the models on the datasets.

In [4]:
n_t, n_buffer = 1024, 20
n_c = 3
batch_size = 32

# True HMM model

In [ ]:
for scale in tqdm(["daily", "weekly", "monthly"]):
    key_data_list_true = [f"{n_c}-state-{scale}", f"t-5-{scale}"]
    model_dict_true = {"true": Viterbi_wrapper(*load_hardy_params(scale, n_c))}
    model_fit_many_datas_models(key_data_list_true, "HMM", model_dict_true, None, path, job_id, batch_size, align=False)

In [5]:
n_c = 3
model_hmm_10init = GaussianHMM_model(n_c, n_init=10, init='k-means++', random_state=random_state, n_iter = 300, tol = 1e-4,)
model_hmm_1init = GaussianHMM_model(n_c, n_init=1, init='k-means', random_state=random_state, n_iter = 300, tol = 1e-4,)
model_dict_hmm = {"HMM10init": model_hmm_10init, "HMM1init": model_hmm_1init, }

In [6]:
key_feat_hmm = "HMM"
param_grid_hmm = None

In [10]:
model_fit_many_datas_models(key_data_list_dict[n_c], [key_feat_hmm], model_dict_hmm, param_grid_hmm, path_data, path_estimation, job_id, batch_size)

1/12 combo starts.


/Users/yizhan/mambaforge/envs/regime-gurobi/lib/python3.10/site-packages/hmmlearn/hmm.py:274: RuntimeWarning: invalid value encountered in divide
  self.means_ = ((means_weight * means_prior + stats['obs'])


ValueError: array must not contain infs or NaNs

## Our models

In [6]:
n_c = 3
model_discrete = jump_model(n_c, state_type="discrete", random_state=random_state)
model_cont_mode = jump_model(n_c, state_type="cont", grid_size=.05, mode_loss=True, random_state=random_state)
model_cont_no_mode = jump_model(n_c, state_type="cont", grid_size=.05, mode_loss=False, random_state=random_state)
model_dict = {"discrete": model_discrete, "cont_mode": model_cont_mode, "cont_no_mode": model_cont_no_mode}

In [7]:
# lambd_list = 10 ** np.linspace(0, 8, 9)
lambd_list = 10 ** np.concatenate(([-2.], np.linspace(-1, 5, 13), [6.])) #[-2.] + list(np.linspace(-1, 5, 13)) + [6.]
print(lambd_list)
param_grid = {'jump_penalty': lambd_list}

[1.00000000e-02 1.00000000e-01 3.16227766e-01 1.00000000e+00
 3.16227766e+00 1.00000000e+01 3.16227766e+01 1.00000000e+02
 3.16227766e+02 1.00000000e+03 3.16227766e+03 1.00000000e+04
 3.16227766e+04 1.00000000e+05 1.00000000e+06]


In [8]:
key_feat_zheng = "zheng"

In [9]:
model_fit_many_datas_models(key_data_list_dict[n_c], [key_feat_zheng], model_dict, param_grid, path_data, path_estimation, job_id, batch_size)

1/18 combo starts.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:03<00:00,  3.95it/s]


shape of the saved model params: (32, 12, 15).
shape of the saved labels: (32, 500, 15).
shape of the saved proba: (32, 500, 3, 15).
1/18 combo done. Time of this combo: 0:00:04s. Total time: 0:00:04s.
2/18 combo starts.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [12:23<00:00, 49.56s/it]


shape of the saved model params: (32, 12, 15).
shape of the saved labels: (32, 500, 15).
shape of the saved proba: (32, 500, 3, 15).
2/18 combo done. Time of this combo: 0:12:24s. Total time: 0:12:28s.
3/18 combo starts.


  0%|                                                                                                                                                                                                                                                    | 0/15 [00:29<?, ?it/s]


KeyboardInterrupt: 